##  Advanced Classification Hackathon

    Author: Stephen Mutiso
    Date: 24 June 2023
    Email: mutisosteve05@gmail.com
    Position: Data Scientist
    Company: Explore-AI


South Africa is a multicultural society that is characterised by its rich linguistic diversity. Language is an indispensable tool that can be used to deepen democracy and also contribute to the social, cultural, intellectual, economic and political life of the South African society.

The country is multilingual with 11 official languages, each of which is guaranteed equal status. Most South Africans are multilingual and able to speak at least two or more of the official languages.

In this challenge, we will take text which is in any of South Africa's 11 Official languages and identify which language the text is in. This is an example of NLP's Language Identification, the task of determining the natural language that a piece of text is written in

In [99]:
import re
import string

import pandas as pd
import numpy as np

In [100]:
data = pd.read_csv('train_set.csv', encoding='UTF-8')
test = pd.read_csv('test_set.csv',encoding='UTF-8')


In [101]:
data.head()


,lang_id,text
0,xho,umgaqo-siseko wenza amalungiselelo kumaziko ax...
1,xho,i-dha iya kuba nobulumko bokubeka umsebenzi na...
2,eng,the province of kwazulu-natal department of tr...
3,nso,o netefatša gore o ba file dilo ka moka tše le...
4,ven,khomishini ya ndinganyiso ya mbeu yo ewa maana...


In [102]:
data_copy = data.copy()


### Data Preprocessing

In [103]:
data.lang_id.describe()


count     33000
unique       11
top         xho
freq       3000
Name: lang_id, dtype: object

In [104]:
data_copy['length'] = data_copy['text'].apply(lambda x: len(x))

In [1]:
data.describe()

NameError: name 'data' is not defined

### Data Cleaning

The function below removes punctuation and ASCII code character using string.punctuation function  
Args:

    data: pandas dataframe

Return:

    Dataframe: clean tweets

In [106]:
def punctuation_remover(x):
    
    x = re.sub(r'[-]',' ',x)
    x = re.sub(r'[_]', ' ', x)
    x = re.sub(r'[^\w\s]','',x)
    x = re.sub('[0-9]+', '', x)
    x = re.sub(r'[^\x00-\x7f]',r'', x)
    return x


#Apply the function to the dataset
data_copy['clean_punc'] = data_copy['text'].apply(punctuation_remover)


In [107]:
def lower(x):

    return x.lower()

data_copy['lower'] = data_copy['clean_punc'].apply(lower)


In [108]:
from nlppreprocess import NLP
nlp = NLP()
nlp.process('couldnt')

'could not'

In [109]:
import nltk
nltk.download('stopwords')


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\mutis\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

The function in the code cell below removes stop-words in the dataset to reduce noise.

Args:
    data: pandas dataframe
Return:
    df:non-stop word

In [110]:
stopword = nltk.corpus.stopwords.words('english')

def remove_stopwords(x):
    
    stopwords = NLP(replace_words=True, remove_stopwords=True, 
                            remove_numbers=True, remove_punctuations=False) 
    x = stopwords.process(x)
    return x
    
data_copy['Text_nonstop'] = data_copy['lower'].apply(lambda x: remove_stopwords(x))


In [111]:
def analyzer (x):
    """
    Function combines all the cleaning operations
    """
    x = punctuation_remover(x)
    x = lower(x)
    x = remove_stopwords(x)
    return x


In [112]:
data_copy.head()

,lang_id,text,length,clean_punc,lower,Text_nonstop
0,xho,umgaqo-siseko wenza amalungiselelo kumaziko ax...,220,umgaqo siseko wenza amalungiselelo kumaziko ax...,umgaqo siseko wenza amalungiselelo kumaziko ax...,umgaqo siseko wenza amalungiselelo kumaziko ax...
1,xho,i-dha iya kuba nobulumko bokubeka umsebenzi na...,252,i dha iya kuba nobulumko bokubeka umsebenzi na...,i dha iya kuba nobulumko bokubeka umsebenzi na...,dha iya kuba nobulumko bokubeka umsebenzi naph...
2,eng,the province of kwazulu-natal department of tr...,264,the province of kwazulu natal department of tr...,the province of kwazulu natal department of tr...,province kwazulu natal department transport in...
3,nso,o netefatša gore o ba file dilo ka moka tše le...,217,o netefata gore o ba file dilo ka moka te le d...,o netefata gore o ba file dilo ka moka te le d...,o netefata gore o ba file dilo ka moka te le d...
4,ven,khomishini ya ndinganyiso ya mbeu yo ewa maana...,239,khomishini ya ndinganyiso ya mbeu yo ewa maana...,khomishini ya ndinganyiso ya mbeu yo ewa maana...,khomishini ya ndinganyiso ya mbeu yo ewa maana...


In [113]:
data['cleaned'] = data['text'].apply(analyzer)
test['cleaned'] = test['text'].apply(analyzer)
 

## Data Preprocessing 

### Text Engineering

In [114]:
# Splitting  features(X) and response(Y) variables
X = data['cleaned']
y = data['lang_id']


In [115]:
from sklearn.model_selection import train_test_split

X_train ,X_test ,y_train ,y_test = train_test_split(X , y, stratify=y,
                                                       test_size =0.4, 
                                                       random_state=42)


### Importing relevant models

In [116]:
# importing Models from scikit-learn

from sklearn.model_selection import KFold
from sklearn.linear_model import LogisticRegression, SGDClassifier, RidgeClassifier
from sklearn.model_selection import cross_val_score, GridSearchCV, RepeatedStratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC, SVC
from sklearn.naive_bayes import MultinomialNB, ComplementNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold


In [117]:
classifiers = [LogisticRegression(random_state =42 , max_iter=5000) , 
       MultinomialNB(), LinearSVC(random_state=42), 
       SGDClassifier(random_state=42), RidgeClassifier(random_state=42)]


In [118]:
from sklearn import metrics
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer , TfidfVectorizer


In [119]:
def performace_assesment(*args , **kwargs):

  model_statistics = {}
  for clf in classifiers:
    model = Pipeline([('tfidf', TfidfVectorizer(stop_words = 'english', max_df=0.9, ngram_range=(1, 5), analyzer= 'char')),
                      ('clf' , clf)
                      ])
    
    model.fit(X_train, y_train) #Training
    model_pred = model.predict(X_test) #Testing

    # Dictionary of Models Performances
    model_statistics[clf.__class__.__name__] = {
        'F1-Macro':metrics.f1_score(y_test, model_pred, average='macro'),
        'F1-Accuracy':metrics.f1_score(y_test, model_pred, average='micro'),
        'F1-Weighted':metrics.f1_score(y_test, model_pred, average='weighted')}
  return pd.DataFrame.from_dict(model_statistics, orient='index')


In [120]:
model_performance = performace_assesment(classifiers , X_train , X_test , y_train , y_test)
model_performance.to_csv('performance.csv')
df = pd.read_csv('performance.csv', index_col = 0)
df.sort_values('F1-Weighted', ascending=False)


,F1-Macro,F1-Accuracy,F1-Weighted
MultinomialNB,0.999394,0.999394,0.999394
RidgeClassifier,0.999167,0.999167,0.999167
LinearSVC,0.999167,0.999167,0.999167
SGDClassifier,0.999091,0.999091,0.999091
LogisticRegression,0.998183,0.998182,0.998183


### Hyperparameter Tuning

In [121]:
def parameter_tuning(*args , **kwargs):
  best_params = {}

  for clf in classifiers:
    model = Pipeline([('tfidf', TfidfVectorizer(stop_words = 'english', max_df=0.9, ngram_range=(1, 5), analyzer= 'char')),
                      ('clf' , clf)])
    model.fit(X_train, y_train) #Training
    
    #Get models performing parameters
    params = model.get_params()
    model_name = clf.__class__.__name__  
    model_name = {}
    for key in params:
      if key.startswith("clf"):
        if len(key) < 5:
          model_name['model'] = params[key]
        else:
            model_name[key[5:]] = params[key]
    best_params[clf.__class__.__name__] = model_name
  return best_params


In [122]:
best_params = parameter_tuning(classifiers, X_train, y_train)

### GridSearchCV

Applying MultinomialNB()

In [123]:
model1 = MultinomialNB()

In [124]:
Vectorize = TfidfVectorizer(stop_words = 'english', max_df=0.9, ngram_range=(1, 5), analyzer= 'char')
X_train = Vectorize.fit_transform(X_train)
X_test = Vectorize.transform(X_test)


In [125]:
stratified_kfold = StratifiedKFold(n_splits=5, shuffle=True,
                                   random_state=42)


In [126]:
best_params[classifiers[1].__class__.__name__]

{'model': MultinomialNB(),
 'alpha': 1.0,
 'class_prior': None,
 'fit_prior': True}

In [127]:
alpha = list(np.linspace(0.1,0.02,4))
param_grid = dict(alpha=alpha)
grid_search = GridSearchCV(estimator= model1,
                           param_grid=param_grid,
                           scoring='f1_weighted',
                           cv=stratified_kfold,
                           error_score=0,
                           n_jobs=-1)


In [128]:
grid_search.fit(X_train, y_train)
prediction = grid_search.predict(X_test)
cv_score = grid_search.best_score_
test_score = grid_search.score(X_test, y_test)


In [129]:
print(f'Cross-validation score: {cv_score}')
print(f'Test score: {test_score}')


Cross-validation score: 0.9996969686252897
Test score: 0.9995455174533855


### Modelling RidgeRegression()

In [130]:
model2 = RidgeClassifier()

In [131]:
best_params[classifiers[4].__class__.__name__]

{'model': RidgeClassifier(random_state=42),
 'alpha': 1.0,
 'class_weight': None,
 'copy_X': True,
 'fit_intercept': True,
 'max_iter': None,
 'normalize': 'deprecated',
 'positive': False,
 'random_state': 42,
 'solver': 'auto',
 'tol': 0.001}

In [132]:
alpha = list(np.linspace(0.15,0.4, 5))
param_grid = dict(alpha=alpha)
grid_search = GridSearchCV(estimator= model2,
                           param_grid=param_grid,
                           scoring='f1_weighted',
                           cv=stratified_kfold,
                           error_score=0,
                           n_jobs=-1)


In [133]:
grid_search.fit(X_train, y_train)
prediction = grid_search.predict(X_test)
cv_score = grid_search.best_score_
test_score = grid_search.score(X_test, y_test)


In [134]:
print(f'Cross-validation score: {cv_score}')
print(f'Test score: {test_score}')
grid_search.best_params_    
grid_search.best_estimator_


Cross-validation score: 0.9994949477404799
Test score: 0.9990909717224429


RidgeClassifier(alpha=0.275)

In [135]:
from sklearn.ensemble import StackingClassifier
from sklearn.pipeline import make_pipeline


In [136]:
X = data['cleaned']
y = data['lang_id']



In [137]:
X_train , X_test , y_train , y_test = train_test_split(X, y,  stratify=y, test_size=0.4, random_state =1)

In [138]:
vect = TfidfVectorizer(stop_words = 'english', max_df=0.9, ngram_range=(2, 6), analyzer= 'char')
X_train = vect.fit_transform(X_train)
X_test = vect.transform(X_test)


In [139]:
multiNB1 = MultinomialNB(alpha=0.1)
multiNB2 = MultinomialNB(alpha=0.1)

estimators = [('multiNB1', multiNB1), ('multiNB2', multiNB2)]
final_est = RidgeClassifier(alpha=0.2125)


In [140]:
stacking_NB2 = StackingClassifier(estimators = estimators,
                           final_estimator = final_est,
                           passthrough = True)


In [141]:
stacking_NB2.fit(X_train , y_train)

StackingClassifier(estimators=[('multiNB1', MultinomialNB(alpha=0.1)),
                               ('multiNB2', MultinomialNB(alpha=0.1))],
                   final_estimator=RidgeClassifier(alpha=0.2125),
                   passthrough=True)

In [142]:
pred = stacking_NB2.predict(X_test)

In [143]:
model_stats = {}
model_stats[stacking_NB2.__class__.__name__] = {
        'F1-Macro':metrics.f1_score(y_test, pred, average='macro'),
        'F1-Accuracy':metrics.f1_score(y_test, pred, average='micro'),
        'F1-Weighted':metrics.f1_score(y_test, pred, average='weighted')}
pd.DataFrame.from_dict(model_stats, orient='index')


,F1-Macro,F1-Accuracy,F1-Weighted
StackingClassifier,0.999773,0.999773,0.999773


In [144]:
count_vec = CountVectorizer(ngram_range=(3,7), analyzer= 'char')
X_train , X_test , y_train , y_test = train_test_split(X, y, stratify=y,test_size=0.05, random_state =1)
X_train = count_vec.fit_transform(X_train)
X_test = count_vec.transform(X_test)


In [145]:
multiNB1 = MultinomialNB(alpha=0.1)
multiNB2 = MultinomialNB(alpha=0.1)
multiNB3 = MultinomialNB(alpha=0.1)

estimators = [('multiNB1', multiNB1), ('multiNB2', multiNB2), ('multiNB3', multiNB3)]
final_est = RidgeClassifier(alpha=0.2125)


In [146]:
stacking_NB3 = StackingClassifier(estimators = estimators,
                           final_estimator = final_est,
                           passthrough = True) 


In [147]:
stacking_NB3.fit(X_train , y_train)

StackingClassifier(estimators=[('multiNB1', MultinomialNB(alpha=0.1)),
                               ('multiNB2', MultinomialNB(alpha=0.1)),
                               ('multiNB3', MultinomialNB(alpha=0.1))],
                   final_estimator=RidgeClassifier(alpha=0.2125),
                   passthrough=True)

In [148]:
pred = stacking_NB3.predict(X_test)

In [149]:
model_stats = {}

model_stats[stacking_NB3.__class__.__name__] = {
        'F1-Macro':metrics.f1_score(y_test, pred, average='macro'),
        'F1-Accuracy':metrics.f1_score(y_test, pred, average='micro'),
        'F1-Weighted':metrics.f1_score(y_test, pred, average='weighted')}
pd.DataFrame.from_dict(model_stats, orient='index')


,F1-Macro,F1-Accuracy,F1-Weighted
StackingClassifier,1.0,1.0,1.0


### Kaggle Submission

In [150]:
X = test['cleaned']
Vectorize = vect.transform(X)


In [151]:
test['lang_id'] = stacking_NB2.predict(Vectorize)

In [152]:
submission = test[['index', 'lang_id']]
submission.to_csv('Submission.csv',index=False)
submission

,index,lang_id
0,1,tsn
1,2,nbl
2,3,ven
3,4,ssw
4,5,afr
...,...,...
5677,5678,eng
5678,5679,nso
5679,5680,sot
5680,5681,sot


In [153]:
test.cleaned

0       mmasepala fa maemo kgethegileng letlelela kgat...
1       uzakwaziswa ngokufaneleko nakungafuneka eminye...
2               tshivhumbeo tshi fana na ngano dza vhathu
3       kube inja nelikati betingevakala kutsi titsini...
4                            winste op buitelandse valuta
                              ...                        
5677                      you mark your ballot in private
5678    ge o ka kgetha ka bowena go se omie mofani ka ...
5679    e ka kopo etsa kgetho ya hao ka hloko hobane h...
5680    tb ke bokudi ba pmb mme morero o tla lefella t...
5681                 vakatjhela iwebhusayidi yethu ku www
Name: cleaned, Length: 5682, dtype: object